This is the main file which consolidates all the activities:
- 

### OPS Website Scraping

This is the script from crawlign the OPS website to get the information such as:
- Dates of the meetings
- Remarks whether it is a regular meeting or not
- Identification whether there is an attachment or not

In [1]:
# import necessary libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


# function to extract html document from given url
def getHTMLdocument(url):
	
	# request for HTML document of given url
	response = requests.get(url)
	
	# response will be provided in JSON format
	return response.text



"""
This part is to populate the data from OPS homepage. 
"""

Meetings = pd.DataFrame(columns = ['DATE', 'LOCATION', 'TIME', 'TYPE'])

#
year_start = 2016
year_end = 2022

for year in range(year_start,year_end+1):
    # assign required credentials
    # assign URL
    url_to_scrape = "https://ottawapoliceboard.ca/opsb-cspo/" + str(year) + "-meetings.html"

    # create document
    html_document = getHTMLdocument(url_to_scrape)

    # create soap object
    soup = BeautifulSoup(html_document, 'html.parser')
    
    """ Note: Modify the code that extracts and cleans the table rows (tr) 
    as per the structure of the HTML of the website they want to scrape. """
    
    """ Note: Modify the code that appends the data to the final data as per the data structure
    of the website they want to scrape. """

    all_raw = soup.find_all('tr')
    all_raw = [x.text for x in all_raw]
    all_raw

    all_uncleaned = []
    for a in all_raw:
        clean_string = re.sub('\n','   ',a).strip()
        all_uncleaned.append(clean_string)

    all_cleaned = []
    for y in all_uncleaned:
        x = re.split("\s{3,}",y)
        all_cleaned.append(x)
    
    for z in all_cleaned:
        while len(z) < 4:
            z.append(" ")
            
    final_df =[]
    try:
        for z in all_cleaned:
            if len(z) == 4:
                final_df.append(z)
    except:
        continue

    meetings = pd.DataFrame(final_df[1:], columns = ["DATE", "LOCATION", "TIME", "TYPE"])

    links =[]
    for sample in meetings['DATE']:
        try:
            link_string = soup.find('tbody').find('a', text = sample).get('href')
            links.append(link_string)
        except: 
            links.append("No Link")

    meetings['LINKS'] = links
    
    Meetings =+ pd.concat([Meetings, meetings])
    
# add meeting ID here
    RegularMeetings = Meetings.loc[Meetings['TYPE'] == 'Regular Meeting'].reset_index().drop(['index'], axis=1)


"""
This part if for getting the unique identifier of each regular meeting. 
Based on this identifier which is the itemid, we will eventually identify which meetings has an attachment.

"""
MeetingID = []
for links in RegularMeetings['LINKS']:
    try:
        url_to_scrape = links
        # create document
        html_document = getHTMLdocument(url_to_scrape)
        # create soup object
        soup = BeautifulSoup(html_document, 'html.parser')

        try:
            """
            There are different text values on the tag for the Chief's verbal. 
            It is necessary to consider all the conditions to make sure that we getting the unique MeetingID.
            """            
            """Note: there will be different tags if needs to extract fields other than the Chief's Verbal"""
            
            CVtag = soup.find_all('a', text="CHIEF’S VERBAL REPORT")
            itemid = CVtag[0]['itemid']
            MeetingID.append(itemid)
        except:
            try:
                CVtag = soup.find_all('a', text="CHIEF'S VERBAL REPORT")
                itemid = CVtag[0]['itemid']
                MeetingID.append(itemid)
            except:
                try:
                    CVtag = soup.find('a', text = "Chief’s verbal report")
                    link_string = CVtag.get('href')
                    itemid = re.findall(r'([0-9][0-9])', link_string)
                    MeetingID.append(itemid[0])
                except:
                    try:
                        CVtag = soup.find('a', text = "CHIEF’S VERBAL REPORT")
                        link_string = CVtag.get('href')
                        itemid = re.findall(r'([0-9][0-9])', link_string)
                        MeetingID.append(itemid[0])
                    except:
                        try:
                            CVtag = soup.find_all('a', text="CHIEF'S VERBAL REPORT")
                            itemid = CVtag[0]['itemid']
                            MeetingID.append(itemid)
                        except:
                            try:
                                CVtag = soup.find_all('a', class_= "SelectableItem")
                                itemid = CVtag[3]['itemid']
                                MeetingID.append(itemid)
                            except:
                                try:
                                    CVtag = soup.find_all('span', text="CHIEF'S VERBAL REPORT")
                                    itemid = CVtag[0]['itemid']
                                    MeetingID.append(itemid)
                                except: 
                                    MeetingID.append('NoID')
    except:
        MeetingID.append('Cancelled')
        
        
RegularMeetings['MeetingID'] = MeetingID


"""
This part loops in all the regular meetings to check the existence of the chief's verbal. 
"""

FileName = []

"""Note: This code too needs to be modified according to the a tag from which the data needs to be extracted """

for (links,id_no) in zip(RegularMeetings['LINKS'], RegularMeetings['MeetingID']):
    if links != '#':
        url_to_scrape = links
        # create document
        html_document = getHTMLdocument(url_to_scrape)
        # create soup object
        soup = BeautifulSoup(html_document, 'html.parser')
        
        try:
            class_name = "AgendaItemAttachment AgendaItemAttachment"+id_no
            CVExistence = soup.find('div', class_=class_name).find('a')
            CVFilename = [x.text for x in CVExistence]
            CVFilename = [''.join(CVFilename)]
            clean_string = re.sub('\n','',CVFilename[0]).strip()
            FileName.append(clean_string)
        except:
            FileName.append("No Attachment")
    else:
        FileName.append("Cancelled Meeting")
        
RegularMeetings['FileName'] = FileName

#Saving a report in the local folder
RegularMeetings.to_csv("RegularMeetings.csv", index=False)

PDFLink = []
for (links,id_no) in zip(RegularMeetings['LINKS'], RegularMeetings['MeetingID']):
    if links != '#':
        url_to_scrape = links
        # create document
        html_document = getHTMLdocument(url_to_scrape)
        # create soup object
        soup = BeautifulSoup(html_document, 'html.parser')
        
        try:
            class_name = "AgendaItemAttachment AgendaItemAttachment"+id_no
            CVExistence = soup.find('div', class_=class_name).find('a')
            CV = CVExistence.get('href')
            CVPDF = [x for x in CV]
            CVFilePDF = [''.join(CVPDF)]
            l = "https://pub-ottawa.escribemeetings.com/"+CVFilePDF[0]
            
            PDFLink.append(l)                      
        except:
            PDFLink.append("No PDF")
    else:
        PDFLink.append("Cancelled Meeting")


RegularMeetings['PDFLink'] = PDFLink
RegularMeetings.to_csv("RegularMeetings.csv", index=False) 

### Sending an Automated Email with the CV Report Attachment

In [ ]:
#Working Automated Email with Attachment

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

sender_email = 'bisitest2023@gmail.com'
receiver_email = ''
password = 'kobvmvusacbvylco'

message = MIMEMultipart()
message["Subject"] = "OPS Web Scraping Chief's Verbals Attachments"
message["From"] = "NoReply@BISITest"
message["To"] = receiver_email

#Note: User can write the message or plain-text as per their requirement for sending an email.

# Create the plain-text and HTML version of your message
text = """\
Hi,

Please refer to the attached file for the copy of Regular Meetings and the corresponding Chief's Verbal Attachment.

From Group 5:
    Fu, Jin
    Ganatra, Ozal
    Marzo, Gilbert
    Varghese, Alin Mary
    Zhong, Vivienne
"""
#Note: User can attach the any files for an attachment in the filename and attachment variable.

message.attach(MIMEText(text, 'plain'))
filename = "ChiefsVerbalReport.csv"
attachment = open("ChiefsVerbalReport.csv", "rb")

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

message.attach(part)


#Note: User can change the mail server as per the requirement. for example, below we have used GMAIL(465)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )

### Downloading the PDF Files and saving it in a local folder

In [2]:
import os
import pandas as pd
import requests
import datetime

# Read CSV file
df = pd.read_csv('RegularMeetings.csv')

# Create directory to store PDF files
if not os.path.exists('pdf_files'):
    os.makedirs('pdf_files')

# Loop through each row in the dataframe
for index, row in df.iterrows():
    # Extract PDF link value from the row
    pdf_link = row['PDFLink']

    # Skip row if PDFLink value is "No PDF" or "Cancelled Meeting"
    if pdf_link in ["No PDF", "Cancelled Meeting"]:
        continue

    # Extract filename from the URL
    filename = pdf_link.split('/')[-1].split('?')[0]

    # Extract date from the DATE column and convert to required format
    date = datetime.datetime.strptime(row['DATE'], '%B %d, %Y').strftime('%Y-%m-%d')

    try:
        # Download PDF file
        response = requests.get(pdf_link, stream=True)
        response.raise_for_status()

        # Save PDF file in the pdf_files directory with corresponding date subfolder
        if not os.path.exists(os.path.join('pdf_files', date)):
            os.makedirs(os.path.join('pdf_files', date))
        with open(os.path.join('pdf_files', date, filename), 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)

        print(f'Successfully downloaded {filename} to pdf_files/{date}')
    except Exception as e:
        print(f'Error downloading {filename} from {pdf_link}: {e}')

Successfully downloaded filestream.ashx to pdf_files/2016-01-25
Successfully downloaded filestream.ashx to pdf_files/2016-03-21
Successfully downloaded filestream.ashx to pdf_files/2016-04-25
Successfully downloaded filestream.ashx to pdf_files/2016-05-30
Successfully downloaded filestream.ashx to pdf_files/2016-07-25
Successfully downloaded filestream.ashx to pdf_files/2016-10-24
Successfully downloaded filestream.ashx to pdf_files/2016-11-28
Successfully downloaded filestream.ashx to pdf_files/2016-11-28
Successfully downloaded filestream.ashx to pdf_files/2016-12-19
Successfully downloaded filestream.ashx to pdf_files/2017-01-23
Successfully downloaded filestream.ashx to pdf_files/2017-02-27
Successfully downloaded filestream.ashx to pdf_files/2017-03-27
Successfully downloaded filestream.ashx to pdf_files/2017-04-24
Successfully downloaded filestream.ashx to pdf_files/2017-05-29
Successfully downloaded filestream.ashx to pdf_files/2017-06-26
Successfully downloaded filestream.ashx 

### Creating Text DataFrame

Looping through all the the pdf files and storing in a dataframe that contains both the data and the full text of the pdf.

In [3]:
import re
import pandas as pd
import urllib
import io
import nltk
import requests
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')
import fitz

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alinm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alinm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alinm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv("RegularMeetings.csv")

In [5]:
raw_text = []

for links in df['PDFLink']:
#for links in url:
    if links in ['Cancelled Meeting','No PDF']:
        raw_text.append('No Text')
    else:
        url = links
        response = requests.get(url)

        response = urllib.request.urlopen(url).read()

        # Extract the text from the PDF
        with io.BytesIO(response) as data:
            reader = fitz.open(stream=data, filetype="pdf")
            text = ""
            for page in reader:
                text += page.get_text()    
        raw_text.append(text)

In [6]:
raw_df = pd.DataFrame(raw_text, columns = ["Raw_Text"])

date = df['DATE']
raw_df['Date'] = date

In [7]:
# This is for cleaning the raw texts in the raw_df dataframe. 

#Note: User can use different type of regular expressions based on the requirement to clean the data.

def clean_text(text):
    
    # Converting texts to lowercase
    text = text.lower()

    # Removing "\n"
    text = re.sub(r'\n', '  ', text)
    
    # Remove double or more spaces
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

In [8]:
# This is for cleaning the raw texts in the raw_df dataframe. 
cleaned_texts = []
for text in raw_df['Raw_Text']:
    text = clean_text(text)
    cleaned_texts.append(text)

raw_df['Cleaned_Text'] = cleaned_texts

In [9]:
raw_df

,Raw_Text,Date,Cleaned_Text
0,"Chief’s Verbal Report \nMonday, January 25, 20...","January 25, 2016","chief’s verbal report monday, january 25, 2016..."
1,Chief Charles Bordeleau \nVerbal Report \nMarc...,"March 21, 2016",chief charles bordeleau verbal report march 21...
2,"Verbal Report \nApril 25, 2016 \nChief Bordele...","April 25, 2016","verbal report april 25, 2016 chief bordeleau g..."
3,"Chief Bordeleau’s Verbal Report \nMay 30, 2016...","May 30, 2016","chief bordeleau’s verbal report may 30, 2016 g..."
4,"Chief’s Verbal Report \nMonday, July 25, 2016 ...","July 25, 2016","chief’s verbal report monday, july 25, 2016 ot..."
...,...,...,...
66,1 \n \nChief Bell’s Verbal Report for OPSB Mee...,"June 27, 2022",1 chief bell’s verbal report for opsb meeting ...
67,No Text,"July 25, 2022",no text
68,1 \n \nI/Chief Bell’s Verbal for OPSB meeting ...,"September 26, 2022",1 i/chief bell’s verbal for opsb meeting septe...
69,No Text,"October 31, 2022",no text


In [10]:
cleaned_df = raw_df[['Date','Cleaned_Text']]
cleaned_df.to_excel('CleanedDF.xlsx',index=False)

### Splitting Full Text to Stokens

In [11]:
# This is for the creation of the dataframe that contains all the tokenized sentences from all the files.

stokens_df = pd.DataFrame()

for date, text in zip(raw_df['Date'],raw_df['Cleaned_Text']):
    stokens = nltk.sent_tokenize(text)

    sent_df = pd.DataFrame(stokens, columns = ['Sentences'])
    sent_df['Date'] = date
    sent_df = sent_df[['Date','Sentences']]
    
    stokens_df =+ pd.concat([stokens_df, sent_df])

In [12]:
#Storing stokens in a dataframe
stokens_df = stokens_df.reset_index(drop=True)

In [13]:
stokens_df

,Date,Sentences
0,"January 25, 2016","chief’s verbal report monday, january 25, 2016..."
1,"January 25, 2016",congratulations mr. chair and vice chair on yo...
2,"January 25, 2016",thank you to you both for your continued leade...
3,"January 25, 2016","for my verbal report, i’d like to make a few b..."
4,"January 25, 2016",we have increased staffing to our guns and gan...
...,...,...
3455,"September 26, 2022","instead of grabbing him, he took his arm and t..."
3456,"September 26, 2022",the conversation appeared to calm the man down.
3457,"September 26, 2022",i was extremely impressed with the way this wa...
3458,"October 31, 2022",no text


In [14]:
#This function gets the sequence of the sentences from the full text for each date (pdf file)
def get_line_sequence(date):
    sequences = []
    line = 0
    for lines in stokens_df.loc[stokens_df['Date'] == date]['Sentences']:
        line += 1
        sequences.append(line)
    return sequences

In [15]:
# Storing stokens and sequence in a single dataframe
stokens_seq = pd.DataFrame(columns = ['Date','Sentences','Sequence'])

for date in raw_df['Date']:
    filtered = stokens_df[stokens_df['Date'] == date].copy()

    seq = []
    line = 0
    for i in filtered['Sentences']:
        line += 1
        seq.append(line)
    
    filtered['Sequence'] = seq
    
    stokens_seq = pd.concat([stokens_seq, filtered])

In [16]:
stokens_seq

,Date,Sentences,Sequence
0,"January 25, 2016","chief’s verbal report monday, january 25, 2016...",1
1,"January 25, 2016",congratulations mr. chair and vice chair on yo...,2
2,"January 25, 2016",thank you to you both for your continued leade...,3
3,"January 25, 2016","for my verbal report, i’d like to make a few b...",4
4,"January 25, 2016",we have increased staffing to our guns and gan...,5
...,...,...,...
3455,"September 26, 2022","instead of grabbing him, he took his arm and t...",82
3456,"September 26, 2022",the conversation appeared to calm the man down.,83
3457,"September 26, 2022",i was extremely impressed with the way this wa...,84
3458,"October 31, 2022",no text,1


### Categorizing sentences [Metrics, Counter Measure, Neither]

Categorizing the stokens based on the following:
1. Metrics
2. Counter Measure
3. Neither

In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
#Importing manually labeled sentences.
labeled_df = pd.read_excel("labeled_data.xlsx")

#Storing input sentence in labeled_df as a list
labeled_data = list(labeled_df['Input Sentence'])

In [20]:
labeled_df.head()

,Input Sentence,Tags
0,"just last weekend , two individuals were charg...",metric
1,"in fact , three armed suspects , a violent per...",metric
2,"earlier this month , central district officers...",metric
3,arrest on attempted murder charges this past w...,countermeasure
4,central district investigators launched an inv...,countermeasure


In [21]:
def get_category(sentence):
    labeled_data = list(labeled_df['Input Sentence'])
    
    labeled_data.append(sentence)
    
    idx = labeled_data.index(sentence)

    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(labeled_data)

    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    similarity_score = [round(i[1],3) for i in sim_scores]
    similarity_score = set(similarity_score)    
    similarity_score.remove(1)
    similarity_score.remove(0)
    max_sim_score = max(set(similarity_score))

    for index,sim_score in sim_scores:
        if round(sim_score,3) == max_sim_score:
            try:
                loc_auto = labeled_df.index[labeled_df['Input Sentence']==str(labeled_data[index])].tolist()
                autotag = labeled_df['Tags'][loc_auto [0]]

                sim_sent = str(labeled_data[index])
                score_sim = str(round(sim_score,3))

                result = pd.DataFrame(columns = ['Input Sentence', 'Similar Sentence', 'Score', 'AutoTag'])

                input_sentence = []
                input_sentence.append(sentence)

                similar_sentence = []
                similar_sentence.append(sim_sent)

                score = []
                score.append(score_sim)

                atags = []
                atags.append(autotag)

                result['Input Sentence'] = input_sentence
                result['Similar Sentence'] = similar_sentence
                result['Score'] = score
                result['AutoTag'] = atags

                return result
            
            except:
                continue
                                    
        else:
            continue

In [22]:
stokens_seq

,Date,Sentences,Sequence
0,"January 25, 2016","chief’s verbal report monday, january 25, 2016...",1
1,"January 25, 2016",congratulations mr. chair and vice chair on yo...,2
2,"January 25, 2016",thank you to you both for your continued leade...,3
3,"January 25, 2016","for my verbal report, i’d like to make a few b...",4
4,"January 25, 2016",we have increased staffing to our guns and gan...,5
...,...,...,...
3455,"September 26, 2022","instead of grabbing him, he took his arm and t...",82
3456,"September 26, 2022",the conversation appeared to calm the man down.,83
3457,"September 26, 2022",i was extremely impressed with the way this wa...,84
3458,"October 31, 2022",no text,1


In [23]:
result = []
date =[]
count = 0
for i, sent in zip(stokens_seq['Date'],stokens_seq['Sentences']):
    try:
        result.append(get_category(sent))
        date.append(i)
        count = count + 1
    except:
        continue
output = pd.concat(result)

In [24]:
output = output.reset_index(drop=True)

In [25]:
output['Date'] = date

In [26]:
sent_tokens = stokens_seq.rename(columns={'Sentences': 'Input Sentence'})

In [27]:
sent_tokens

,Date,Input Sentence,Sequence
0,"January 25, 2016","chief’s verbal report monday, january 25, 2016...",1
1,"January 25, 2016",congratulations mr. chair and vice chair on yo...,2
2,"January 25, 2016",thank you to you both for your continued leade...,3
3,"January 25, 2016","for my verbal report, i’d like to make a few b...",4
4,"January 25, 2016",we have increased staffing to our guns and gan...,5
...,...,...,...
3455,"September 26, 2022","instead of grabbing him, he took his arm and t...",82
3456,"September 26, 2022",the conversation appeared to calm the man down.,83
3457,"September 26, 2022",i was extremely impressed with the way this wa...,84
3458,"October 31, 2022",no text,1


In [28]:
output_sequenced = pd.merge(sent_tokens, output, on=['Date', 'Input Sentence'], how = 'left')

In [29]:
output_sequenced = output_sequenced.drop_duplicates().reset_index()

### This section tags each stokens based on the crime mentioned.

In [30]:
#Initiating a crimes dictionary that contains all general crimes and terms that is associated to that.

#Note: User can remove or update the crimes based on the requirements

crimes = {
        'homicide' : ['murder','infanticide', 'kill', 'homicide', 'manslaughter'],
        'stealing' : ['rob', 'steal', 'theft'],
        'assault': ['assault', 'battery', 'attack', 'harm'],
        'drug' : ['cannabis','cocaine','cannabis','drug trafficking', 'fentanyl', 'controlled drugs','production drug', 'distribution drug', 'methamphetamines', 'crystal meth', 'methylenedioxyamphetamine', 'ecstasy', 'opioid', 'drug']
        }

In [31]:
dates = []
sentences =[]
tags = []
categories = []

for i in range(0,len(output_sequenced)):
    for category in crimes.keys():
        for terms in crimes[category]:
            if terms in output_sequenced['Input Sentence'][i]:
                date = output_sequenced['Date'][i]
                sentence = output_sequenced['Input Sentence'][i]
                dates.append(date)
                sentences.append(sentence)
                tags.append(terms)
                categories.append(category)
                
            else:
                continue

In [32]:
#Storing the the result in a data frame
tagged_df = pd.DataFrame()
tagged_df['Date'] = dates
tagged_df['Input Sentence'] = sentences
tagged_df['Tag'] = tags
tagged_df['Category'] = categories

In [33]:
output_df = output_sequenced.fillna(0)

In [34]:
final_df = pd.merge(output_df, tagged_df, how='left', on=['Date', 'Input Sentence'])

In [35]:
final_df.to_excel('final_dataframe.xlsx', index=False)

## GUI for getting the information (Date and Sentences) based on Keywords

In [36]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd

In [37]:
regular_df = pd.read_csv('RegularMeetings.csv')
regular_df = regular_df[['DATE','PDFLink']]
regular_df

,DATE,PDFLink
0,"January 25, 2016",https://pub-ottawa.escribemeetings.com/filestr...
1,"March 21, 2016",https://pub-ottawa.escribemeetings.com/filestr...
2,"April 25, 2016",https://pub-ottawa.escribemeetings.com/filestr...
3,"May 30, 2016",https://pub-ottawa.escribemeetings.com/filestr...
4,"July 25, 2016",https://pub-ottawa.escribemeetings.com/filestr...
...,...,...
66,"June 27, 2022",https://pub-ottawa.escribemeetings.com/filestr...
67,"July 25, 2022",No PDF
68,"September 26, 2022",https://pub-ottawa.escribemeetings.com/filestr...
69,"October 31, 2022",No PDF


In [38]:
import requests
import re
import pandas as pd
import urllib
import io
import nltk
import requests
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')
import fitz

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alinm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alinm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alinm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
cleaned_df = raw_df[['Date','Cleaned_Text']]
cleaned_df.to_excel('CleanedDF.xlsx',index=False)

In [40]:
# Read the Excel file
text_df = pd.read_excel("CleanedDF.xlsx")
text_df

,Date,Cleaned_Text
0,"January 25, 2016","chief’s verbal report monday, january 25, 2016..."
1,"March 21, 2016",chief charles bordeleau verbal report march 21...
2,"April 25, 2016","verbal report april 25, 2016 chief bordeleau g..."
3,"May 30, 2016","chief bordeleau’s verbal report may 30, 2016 g..."
4,"July 25, 2016","chief’s verbal report monday, july 25, 2016 ot..."
...,...,...
66,"June 27, 2022",1 chief bell’s verbal report for opsb meeting ...
67,"July 25, 2022",no text
68,"September 26, 2022",1 i/chief bell’s verbal for opsb meeting septe...
69,"October 31, 2022",no text


In [41]:
text_df = text_df.rename(columns={'Date': 'DATE'})
dataframe = pd.merge(regular_df,text_df, on =['DATE'])
dataframe

,DATE,PDFLink,Cleaned_Text
0,"January 25, 2016",https://pub-ottawa.escribemeetings.com/filestr...,"chief’s verbal report monday, january 25, 2016..."
1,"March 21, 2016",https://pub-ottawa.escribemeetings.com/filestr...,chief charles bordeleau verbal report march 21...
2,"April 25, 2016",https://pub-ottawa.escribemeetings.com/filestr...,"verbal report april 25, 2016 chief bordeleau g..."
3,"May 30, 2016",https://pub-ottawa.escribemeetings.com/filestr...,"chief bordeleau’s verbal report may 30, 2016 g..."
4,"July 25, 2016",https://pub-ottawa.escribemeetings.com/filestr...,"chief’s verbal report monday, july 25, 2016 ot..."
...,...,...,...
68,"June 27, 2022",https://pub-ottawa.escribemeetings.com/filestr...,1 chief bell’s verbal report for opsb meeting ...
69,"July 25, 2022",No PDF,no text
70,"September 26, 2022",https://pub-ottawa.escribemeetings.com/filestr...,1 i/chief bell’s verbal for opsb meeting septe...
71,"October 31, 2022",No PDF,no text


In [42]:
df = dataframe[['DATE','Cleaned_Text', 'PDFLink']]
df.to_excel('CleanedDF_with_Links.xlsx',index=False)

In [43]:
filtered = pd.read_excel("final_dataframe.xlsx")

In [44]:
df = df.rename(columns={'DATE':'Date'})
cleaned_WLink = pd.merge(filtered,df, on =['Date'])
cleaned_WLink

,index,Date,Input Sentence,Sequence,Similar Sentence,Score,AutoTag,Tag,Category,Cleaned_Text,PDFLink
0,0,"January 25, 2016","chief’s verbal report monday, january 25, 2016...",1,"chief’s verbal report monday , november 28, 20...",0.535,neither,NaN,NaN,"chief’s verbal report monday, january 25, 2016...",https://pub-ottawa.escribemeetings.com/filestr...
1,1,"January 25, 2016",congratulations mr. chair and vice chair on yo...,2,mr . ibrahim was murdered in february .,0.177,metric,NaN,NaN,"chief’s verbal report monday, january 25, 2016...",https://pub-ottawa.escribemeetings.com/filestr...
2,2,"January 25, 2016",thank you to you both for your continued leade...,3,the homicide highlights a continued national t...,0.174,metric,NaN,NaN,"chief’s verbal report monday, january 25, 2016...",https://pub-ottawa.escribemeetings.com/filestr...
3,3,"January 25, 2016","for my verbal report, i’d like to make a few b...",4,arrests in homicide we continue to see success...,0.346,neither,NaN,NaN,"chief’s verbal report monday, january 25, 2016...",https://pub-ottawa.escribemeetings.com/filestr...
4,4,"January 25, 2016",we have increased staffing to our guns and gan...,5,we have also set up a task force of officers f...,0.224,countermeasure,NaN,NaN,"chief’s verbal report monday, january 25, 2016...",https://pub-ottawa.escribemeetings.com/filestr...
...,...,...,...,...,...,...,...,...,...,...,...
3912,4133,"September 26, 2022","instead of grabbing him, he took his arm and t...",82,"as well , between october 9-13, an additional ...",0.144,metric,NaN,NaN,1 i/chief bell’s verbal for opsb meeting septe...,https://pub-ottawa.escribemeetings.com/filestr...
3913,4134,"September 26, 2022",the conversation appeared to calm the man down.,83,"while off duty , an officer intervened on an a...",0.204,countermeasure,NaN,NaN,1 i/chief bell’s verbal for opsb meeting septe...,https://pub-ottawa.escribemeetings.com/filestr...
3914,4135,"September 26, 2022",i was extremely impressed with the way this wa...,84,the man became aggressive with the officer and...,0.267,metric,NaN,NaN,1 i/chief bell’s verbal for opsb meeting septe...,https://pub-ottawa.escribemeetings.com/filestr...
3915,4136,"October 31, 2022",no text,1,0,0.000,0,NaN,NaN,no text,No PDF


In [45]:
cleaned_WLink = cleaned_WLink[(cleaned_WLink['AutoTag'] == 'metric') | (cleaned_WLink['AutoTag'] == 'countermeasure')]
cleaned_WLink.columns

cleaned_WLink = cleaned_WLink[['Date', 'Input Sentence', 'Sequence', 'AutoTag', 'PDFLink']].drop_duplicates()
cleaned_WLink

,Date,Input Sentence,Sequence,AutoTag,PDFLink
1,"January 25, 2016",congratulations mr. chair and vice chair on yo...,2,metric,https://pub-ottawa.escribemeetings.com/filestr...
2,"January 25, 2016",thank you to you both for your continued leade...,3,metric,https://pub-ottawa.escribemeetings.com/filestr...
4,"January 25, 2016",we have increased staffing to our guns and gan...,5,countermeasure,https://pub-ottawa.escribemeetings.com/filestr...
5,"January 25, 2016",we began the year with 5 reported shootings.,6,metric,https://pub-ottawa.escribemeetings.com/filestr...
8,"January 25, 2016",just last week 2 individuals were charged in r...,9,countermeasure,https://pub-ottawa.escribemeetings.com/filestr...
...,...,...,...,...,...
3909,"September 26, 2022",commendation every month we have a commendatio...,79,countermeasure,https://pub-ottawa.escribemeetings.com/filestr...
3911,"September 26, 2022",an unmarked police car stopped and the officer...,81,metric,https://pub-ottawa.escribemeetings.com/filestr...
3912,"September 26, 2022","instead of grabbing him, he took his arm and t...",82,metric,https://pub-ottawa.escribemeetings.com/filestr...
3913,"September 26, 2022",the conversation appeared to calm the man down.,83,countermeasure,https://pub-ottawa.escribemeetings.com/filestr...


In [46]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import webbrowser
import datetime

In [47]:
#create a window
window = tk.Tk()
window.title('PDF search')
window.geometry('1030x460')

#create search label
search_label = ttk.Label(window, text='Enter search term:')
search_label.grid(row=1, column=0)

#create search entry
search_entry = ttk.Entry(window, width=50)
search_entry.grid(row=1, column=1)

#create search button
search_button = ttk.Button(window, text='Search')
search_button.grid(row=0, column=2)

# Create a Button widget to trigger the output generation
download_button = tk.Button(window, text="Download")
download_button.grid(row=2, column=2)

#create text widget to display report results
search_results_text = tk.Text(window, wrap='word', width=30)
search_results_text.grid(row=3, column=0, columnspan=1, padx=10, pady=10)

#create text widget to display search results
report_results_text = tk.Text(window, wrap='word', width=90)
report_results_text.grid(row=3, column=1, columnspan=2, padx=10, pady=10)

#configure hyperlink style
search_results_text.tag_configure('hyperlink', foreground='blue', underline=True)

#search function
def search_pdfs():
    #clear previous search results
    search_results_text.delete('1.0', tk.END)
    
    #get the search term
    search_term = search_entry.get()

    #keep track of displayed dates
    displayed_dates = set()

    def open_pdf_link(event):
        #retrieve the correct PDFLink based on the clicked date hyperlink
        clicked_date = search_results_text.tag_prevrange('hyperlink', event.widget.index('@%s,%s' % (event.x, event.y)))
        clicked_date_text = search_results_text.get(clicked_date[0], clicked_date[1])
        pdf_link = cleaned_WLink[cleaned_WLink['Date'] == clicked_date_text]['PDFLink'].iloc[0]

        #open the PDFLink in the default web browser
        webbrowser.open(pdf_link)

    for texts, dates, pdf_links in zip(cleaned_WLink['Input Sentence'], cleaned_WLink['Date'], cleaned_WLink['PDFLink']):
        if search_term in texts and pdf_links not in ['No PDF', 'Cancelled Meeting']:
            #display date only once
            if dates not in displayed_dates:
                #add date as hyperlink to search results
                search_results_text.insert(tk.END, dates, ('hyperlink',))
                search_results_text.insert(tk.END, '\n')
                search_results_text.tag_bind('hyperlink', '<Button-1>', open_pdf_link)
                #add displayed date to set
                displayed_dates.add(dates)
        else:
            continue
            
def display_report():    
    # clear previous search results
    report_results_text.delete('1.0', tk.END)

    # get the search term
    search_term = search_entry.get()

    # search for the keyword in the 'OG sentences' column and only select rows with 'metric' or 'countermeasure' in the 'Auto Tag' column
    sentences_with_keyword = cleaned_WLink[(cleaned_WLink['Input Sentence'].str.contains(search_term, case=False)) & (cleaned_WLink['AutoTag'].isin(['metric', 'countermeasure']))]
    
    # sort by date in ascending order
    sentences_with_keyword = sentences_with_keyword.sort_values('Date', ascending=True)

    # group the sentences by date and tag
    sentences_by_date_and_tag = sentences_with_keyword.groupby(['Date', 'AutoTag'])['Input Sentence'].apply(list).to_dict()
    
    # sort the dates in the format of 'Month Day, Year' and print each date and its corresponding sentences that contain the keyword and the 'metric' or 'countermeasure' tag
    sentences_by_date = sentences_with_keyword.groupby('Date').apply(lambda x: x[['AutoTag', 'Input Sentence']].to_dict('records')).to_dict()
    for meeting_date in sorted(sentences_by_date.keys(), key=lambda x: datetime.datetime.strptime(x, '%B %d, %Y')):
        report_date = datetime.datetime.strptime(meeting_date, '%B %d, %Y').strftime('%B %d, %Y')
        report = f"{report_date}:\n\n"
        for tag in ['Metric', 'CounterMeasure']:
            if tag.lower() in [x['AutoTag'].lower() for x in sentences_by_date[meeting_date]]:
                report += f"{tag}:\n\n"
                for sentence in [x['Input Sentence'] for x in sentences_by_date[meeting_date] if x['AutoTag'].lower() == tag.lower()]:
                    report += f"- {sentence}\n\n"
        report_results_text.insert(tk.END, report.encode('utf-8'))
        
def combined_function():
    search_pdfs()
    display_report()
            
# Create a Button widget to download the output
def download_output():
    filename = filedialog.asksaveasfilename(defaultextension=".txt")
    if filename:
        # Get the text to be written to the file
        text = report_results_text.get('1.0', 'end')

        # Encode the text as UTF-8
        text_utf8 = text.encode('utf-8')

        # Open the file in binary mode with UTF-8 encoding
        with codecs.open(filename, 'w', encoding='utf-8') as f:
            # Write the UTF-8 encoded text to the file
            f.write(text_utf8.decode('utf-8'))

#set search button command
search_button.configure(command=combined_function)
download_button.configure(command=download_output)

#run the main loop
window.mainloop()